# Laminar Flow over cylinder (2D)
This notebook implements an unsteady incompressible Navier-Stokes solver for the flow over a cylinder (the problem is based on the [tutorial of FEniCSx](https://jorgensd.github.io/dolfinx-tutorial/chapter2/ns_code2.html), that is the DFG 2D-3 benchmark).

The problem is strong form reads:
\begin{equation}
\left\{
\begin{array}{ll}
    \nabla \cdot \mathbf{u} = 0& in\;\Omega\\
    \displaystyle \frac{\partial \mathbf{u}}{\partial t}+\left(\mathbf{u}\cdot \nabla\right)\mathbf{u}= \nu\Delta \mathbf{u}-\nabla p & in\;\Omega\\ & \\
    \left. \mathbf{u}\right|_{t=0}=\mathbf{0} & in\;\Omega\\ & \\
    \mathbf{u} = \frac{4U(t)\,(y-0.2)\cdot(0.21-y)}{0.41^2}\mathbf{i} & on\;\Gamma_{in}\\
    \mathbf{u} = \mathbf{0} & on\;\Gamma_w\\
    \left(\nu\nabla \mathbf{u}-p\mathbb{I}\right)\cdot \mathbf{n}=\mathbf{g} & on \;\Gamma_{out}
\end{array}
\right.
\end{equation}
in which $\nu = 1e-3$, $\mathbf{g} = \mathbf{0}$ and $U(t) = 1.5 \sin\left(\frac{\pi t}{8}\right)$.

In [1]:
from tqdm import tqdm
import numpy as np

import dolfinx
from mpi4py import MPI
from dolfinx import mesh
from dolfinx.io import gmshio, XDMFFile
from dolfinx import fem
from dolfinx.fem import Function, FunctionSpace, dirichletbc, locate_dofs_topological, form
import ufl
from ufl import grad, div, nabla_grad, dot, inner
from petsc4py import PETSc


gdim = 2
domain, ct, ft = gmshio.read_from_msh("Dfg2D.msh", MPI.COMM_WORLD, rank=0, gdim = gdim)

inl_marker  = 10
out_marker  = 20
walls_marker  = 30
obsta_marker  = 40

dx = ufl.Measure("dx", domain=domain, subdomain_data=ct)
ds = ufl.Measure("ds", domain=domain, subdomain_data=ft)

Info    : Reading 'Dfg2D.msh'...
Info    : 18 entities
Info    : 3645 nodes
Info    : 7290 elements
Info    : Done reading 'Dfg2D.msh'


## Unsteady NS solver
Now, we can define the variational formulations for the time advancement loop. 

A predictor-corrector algorithm is used.

### Predictor
Let's start with the projection step at time $t^{n+1}$
\begin{equation}
\left\{
\begin{array}{ll}
    \nabla \cdot \mathbf{u} = 0& in\;\Omega\\
    \displaystyle \frac{ \mathbf{\tilde{u}} - \mathbf{u}^n}{\Delta t}+\left(\mathbf{u}^{n}\cdot \nabla\right)\mathbf{\tilde{u}}= \nu\Delta \mathbf{\tilde{u}}-\nabla p^n & in\;\Omega\\ & \\
    \left. \mathbf{u}\right|_{t=0}=\mathbf{0} & in\;\Omega\\ & \\
    \mathbf{\tilde{u}} = \frac{4U(t)\,(y-0.2)\cdot(0.21-y)}{0.41^2}\mathbf{i} & on\;\Gamma_{in}\\
    \mathbf{\tilde{u}} = \mathbf{0} & on\;\Gamma_w\\
    \left(\nu\nabla \mathbf{u}-p^n\mathbb{I}\right)\cdot \mathbf{n}=\mathbf{0} & on \;\Gamma_{out}
\end{array}
\right.
\end{equation}
whose weak formulation reads
\begin{equation}
\int_\Omega \tilde{\mathbf{u}}\cdot \mathbf{v}\,d\Omega + \Delta t \int_\Omega \left(\mathbf{u}^n\cdot \nabla\right)\tilde{\mathbf{u}}\cdot \mathbf{v}\,d\Omega + \nu \Delta t \int_\Omega\nabla \tilde{\mathbf{u}}\cdot \nabla \mathbf{v}\,d\Omega =
\int_\Omega {\mathbf{u}}^n\cdot \mathbf{v}\,d\Omega +  \Delta t\int_\Omega p^n\,\nabla\cdot \mathbf{v}\,d\Omega 
\end{equation}

In [2]:
class InletVelocity():
    def __init__(self, U, t):
        self.U = U
        self.t = t

    def __call__(self, x):
        values = np.zeros((gdim, x.shape[1]),dtype=PETSc.ScalarType)
        values[0] = 4 * self.U(self.t) * (x[1]) * (0.41 - x[1]) / 0.41**2
        return values
    
class step01_predict():
    def __init__(self, domain, ft, nu_value = 0.001) -> None:
        self.domain = domain
        self.ft = ft
        self.fdim = self.domain.geometry.dim - 1

        self.nu = fem.Constant(self.domain, PETSc.ScalarType(nu_value))

        self.P2 = ufl.VectorElement("Lagrange", self.domain.ufl_cell(), 2)
        self.V = FunctionSpace(self.domain, self.P2)
        self.P1 = ufl.FiniteElement("Lagrange", self.domain.ufl_cell(), 1)
        self.Q = FunctionSpace(self.domain, self.P1)

        self.u = ufl.TrialFunction(self.V)
        self.v = ufl.TestFunction(self.V)

        self.uOld = Function(self.V)
        self.pOld = Function(self.Q)

    def assignBC(self, inlet, outlet, walls, obstacle):

        self.t = 0
        # self.U = lambda tt: 1.5 * np.sin(np.pi * tt / 8.) 
        self.U = lambda tt: 1. / (1+np.exp(-tt))

        # Inlet BC
        self.inlet_velocity = InletVelocity(self.U, self.t)
        self.u_in = Function(self.V)
        self.u_in.interpolate(self.inlet_velocity)
        bc_in = dirichletbc(self.u_in, locate_dofs_topological(self.V, self.fdim, self.ft.find(inlet)))

        # Walls and obstacle BCs
        self.u_nonslip = np.array((0.,) * self.domain.geometry.dim, dtype=PETSc.ScalarType)
        bc_w   = dirichletbc(self.u_nonslip, locate_dofs_topological(self.V, self.fdim, ft.find(walls)), self.V)
        bc_obs = dirichletbc(self.u_nonslip, locate_dofs_topological(self.V, self.fdim, ft.find(obstacle)), self.V)

        self.bcs = [bc_in, bc_w, bc_obs]

    def assemble(self, dt_value):

        self.dt = fem.Constant(self.domain, PETSc.ScalarType(dt_value))

        self.lhs = (inner(self.u, self.v) * dx +
                    inner(self.dt * dot(self.uOld, nabla_grad(self.u)), self.v) * dx +
                    inner(self.dt * self.nu * grad(self.u), grad(self.v)) * dx ) 
        
        self.rhs = (dot(self.uOld, self.v) - self.dt * dot(grad(self.pOld), self.v)) * dx

        self.a = form(self.lhs)
        self.L = form(self.rhs)

        self.A = fem.petsc.create_matrix(self.a)
        self.b = fem.petsc.create_vector(self.L)

        self.solver = PETSc.KSP().create(self.domain.comm)
        self.solver.setOperators(self.A)
        self.solver.setType(PETSc.KSP.Type.BCGS)
        self.solver.getPC().setType(PETSc.PC.Type.JACOBI)
        # self.solver.setType(PETSc.KSP.Type.PREONLY)
        # self.solver.getPC().setType(PETSc.PC.Type.LU)

    def solve(self, t, uOld, pOld):

        self.uOld.x.array[:] = uOld.x.array[:]
        self.pOld.x.array[:] = pOld.x.array[:]

        self.t = t
        self.inlet_velocity.t = self.t
        self.u_in.interpolate(self.inlet_velocity)

        self.A.zeroEntries()
        fem.petsc.assemble_matrix(self.A, self.a, bcs = self.bcs)
        self.A.assemble()

        with self.b.localForm() as loc:
            loc.set(0)
        fem.petsc.assemble_vector(self.b, self.L)
        fem.petsc.apply_lifting(self.b, [self.a], [self.bcs])
        self.b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(self.b, self.bcs)

        uTilde = Function(self.V).copy()
        self.solver.solve(self.b, uTilde.vector)
        uTilde.x.scatter_forward()

        return uTilde

### Projection
The projection step consists in a Poisson problem, i.e.
\begin{equation}
\left\{
    \begin{array}{ll}
        -\Delta \delta p=-\frac{1}{\Delta t}\nabla\cdot \tilde{\mathbf{u}} & in\;\Omega\\
        \delta p = 0 & on\;\Gamma_{out}\\
        \nabla\delta p\cdot \mathbf{n}=0& on\;\partial\Omega\setminus \Gamma_{out}
    \end{array}
\right.
\end{equation}
whose weak formulation reads
\begin{equation}
\int_\Omega \nabla \delta p\cdot \nabla q\,d\Omega = -\frac{1}{\Delta t}\int_\Omega q\nabla \cdot \tilde{\mathbf{u}}\,d\Omega
\qquad \forall q\in\mathcal{Q}
\end{equation}

In [3]:
class step02_project():
    def __init__(self, domain, ft) -> None:
        self.domain = domain
        self.ft = ft
        self.fdim = self.domain.geometry.dim - 1

        self.P2 = ufl.VectorElement("Lagrange", self.domain.ufl_cell(), 2)
        self.V = FunctionSpace(self.domain, self.P2)
        self.P1 = ufl.FiniteElement("Lagrange", self.domain.ufl_cell(), 1)
        self.Q = FunctionSpace(self.domain, self.P1)

        self.p = ufl.TrialFunction(self.Q)
        self.q = ufl.TestFunction(self.Q)

        self.uTilde = Function(self.V)

    def assignBC(self, inlet, outlet, walls, obstacle):

        self.bcs = [dirichletbc(PETSc.ScalarType(0), locate_dofs_topological(self.Q, self.fdim, ft.find(outlet)), self.Q)]

    def assemble(self, dt_value):

        self.dt = fem.Constant(self.domain, PETSc.ScalarType(dt_value))

        self.lhs = (self.dt * dot(grad(self.p), grad(self.q)) * dx)        
        self.rhs = (- 1. * dot(div(self.uTilde), self.q) * dx)

        self.a = form(self.lhs)
        self.L = form(self.rhs)

        self.A = fem.petsc.assemble_matrix(self.a, bcs=self.bcs)
        self.A.assemble()
        self.b = fem.petsc.create_vector(self.L)

        self.solver = PETSc.KSP().create(self.domain.comm)
        self.solver.setOperators(self.A)
        self.solver.setType(PETSc.KSP.Type.CG)
        self.solver.getPC().setType(PETSc.PC.Type.SOR)
        # self.solver.setType(PETSc.KSP.Type.PREONLY)
        # self.solver.getPC().setType(PETSc.PC.Type.LU)

    def solve(self, u_predict):
        
        self.uTilde.x.array[:] = u_predict.x.array[:]
        
        with self.b.localForm() as loc:
            loc.set(0)
        fem.petsc.assemble_vector(self.b, self.L)
        fem.petsc.apply_lifting(self.b, [self.a], [self.bcs])
        self.b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(self.b, self.bcs)

        deltaP = Function(self.Q).copy()
        self.solver.solve(self.b, deltaP.vector)
        deltaP.x.scatter_forward()

        return deltaP

### Correction
Then, a third auxiliary variational problem is defined to update the velocity
\begin{equation}
\int_\Omega \mathbf{u}^{n+1} \cdot \mathbf{v} \,d\Omega= \int_\Omega\left( \tilde{\mathbf{u}}-\Delta t\nabla\delta p\right) \cdot \mathbf{v} \,d\Omega
\end{equation}

In [4]:
class step03_correct():
    def __init__(self, domain, ft) -> None:
        self.domain = domain
        self.ft = ft
        self.fdim = self.domain.geometry.dim - 1

        self.P2 = ufl.VectorElement("Lagrange", self.domain.ufl_cell(), 2)
        self.V = FunctionSpace(self.domain, self.P2)
        self.P1 = ufl.FiniteElement("Lagrange", self.domain.ufl_cell(), 1)
        self.Q = FunctionSpace(self.domain, self.P1)

        self.u = ufl.TrialFunction(self.V)
        self.v = ufl.TestFunction(self.V)

        self.uTilde = Function(self.V)
        self.deltaP = Function(self.Q)

    def assemble(self, dt_value):

        self.dt = fem.Constant(self.domain, PETSc.ScalarType(dt_value))

        self.lhs = (dot(self.u, self.v) * dx)        
        self.rhs = (dot(self.uTilde - self.dt * grad(self.deltaP), self.v) * dx)

        self.a = form(self.lhs)
        self.L = form(self.rhs)

        self.A = fem.petsc.assemble_matrix(self.a)
        self.A.assemble()
        self.b = fem.petsc.create_vector(self.L)

        self.solver = PETSc.KSP().create(self.domain.comm)
        self.solver.setOperators(self.A)
        self.solver.setType(PETSc.KSP.Type.CG)
        self.solver.getPC().setType(PETSc.PC.Type.SOR)
        # self.solver.setType(PETSc.KSP.Type.PREONLY)
        # self.solver.getPC().setType(PETSc.PC.Type.LU)

    def solve(self, u_predict, deltaP):
        
        self.uTilde.x.array[:] = u_predict.x.array[:]
        self.deltaP.x.array[:] = deltaP.x.array[:]
        
        with self.b.localForm() as loc:
            loc.set(0)
        fem.petsc.assemble_vector(self.b, self.L)
        self.b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)

        u_new = Function(self.V).copy()
        self.solver.solve(self.b, u_new.vector)
        u_new.x.scatter_forward()

        return u_new

Let us assemble all the problems

In [5]:
dt = 1/1600

pred = step01_predict(domain, ft)
pred.assignBC(inl_marker, out_marker, walls_marker, obsta_marker)
pred.assemble(dt_value=dt)

proj = step02_project(domain, ft)
proj.assignBC(inl_marker, out_marker, walls_marker, obsta_marker)
proj.assemble(dt_value=dt)

corr = step03_correct(domain, ft)
corr.assemble(dt_value=dt)

## Time solver

In [6]:
u_sol = Function(pred.V)
p_sol = Function(pred.Q)

u_sol.name = "U"
p_sol.name = "p"

t = 0.
T = 4.
num_steps = int(T/dt)

LL = 16
kk = 0

u_xdmf = XDMFFile(domain.comm, "U.xdmf", "w")
u_xdmf.write_mesh(domain)
u_xdmf.write_function(u_sol, t)

p_xdmf = XDMFFile(domain.comm, "p.xdmf", "w")
p_xdmf.write_mesh(domain)
p_xdmf.write_function(p_sol, t)

for ii in tqdm(range(num_steps)):
    t += dt

    # Solving NS system
    uTilde = pred.solve(t, u_sol, p_sol)
    deltaP = proj.solve(uTilde)
    uNew   = corr.solve(uTilde, deltaP)

    # Update solutions
    u_sol.x.array[:] = uNew.x.array[:]
    u_sol.x.scatter_forward()
    p_sol.vector.axpy(1., deltaP.vector)  
    p_sol.x.scatter_forward()


    if ii % LL == 0:
        u_xdmf.write_function(u_sol, t)  
        p_xdmf.write_function(p_sol, t)

u_xdmf.close()
p_xdmf.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6400/6400 [17:28<00:00,  6.10it/s]
